In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from os import path

In [2]:

template = pd.read_table('bandVague template.txt',delimiter=' ')
means = interp1d(template['phase'],template['mean'],fill_value="extrapolate")
temp1 = interp1d(template['phase'],template['FPC1'],fill_value="extrapolate")
temp2 = interp1d(template['phase'],template['FPC2'],fill_value="extrapolate")
temp3 = interp1d(template['phase'],template['FPC3'],fill_value="extrapolate")
temp4 = interp1d(template['phase'],template['FPC4'],fill_value="extrapolate")
peaklist = pd.read_csv('SNgoodlistpeaks.txt',delim_whitespace=True,header=None)
peaklist.columns=['sn_name','z','w2t0','m2t0','w1t0','ut0','bt0','vt0','w2peak','m2peak','w1peak','upeak','bpeak','vpeak']
finw2 = []
finm2 = []
finw1 = []
finu = []
finb = []
finv = []
nana = [np.nan,np.nan,np.nan,np.nan]
dfw2 = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
dfm2 = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
dfw1 = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
dfu = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
dfb = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
dfv = pd.DataFrame(columns=['SNname','weight1','weight2','weight3','peak_mjd','peak_mag','peak_phase','dM15','resid'])
efw2 = pd.DataFrame(columns=['SNname','pcov','dmstd'])
efm2 = pd.DataFrame(columns=['SNname','pcov','dmstd'])
efw1 = pd.DataFrame(columns=['SNname','pcov','dmstd'])
efu = pd.DataFrame(columns=['SNname','pcov','dmstd'])
efb = pd.DataFrame(columns=['SNname','pcov','dmstd'])
efv = pd.DataFrame(columns=['SNname','pcov','dmstd'])

In [3]:
# Code to run full template fitting on all the SN
def bv(name,fignum):
    print(name)
    global dfw2
    global dfm2
    global dfw1
    global dfu
    global dfb
    global dfv
    global efw2
    global efm2
    global efw1
    global efu
    global efb
    global efv
    if name == 'SN2020hvf':
        sn_data = pd.read_csv('sndata_files/'+name+'_uvotB22.1.dat',delim_whitespace=True,comment='#')
    elif name == 'SN2019np' or name == 'SN2018ilu' or name == 'SN2018aoz':
        sn_data = pd.read_csv('sndata_files/'+name+'_uvotB22.18.dat',delim_whitespace=True,comment='#')
    else:
        sn_data = pd.read_csv('sndata_files/'+name+'_uvotB15.1.dat',delim_whitespace=True,comment='#')
    #sn_data = pd.read_csv('sndata_files/'+name+'_uvotB15.1.dat',delim_whitespace=True,comment='#')
    sn_data.columns=['Filter','MJD','Mag','Magerr','3SigErr','0.98Lim','Rate','RateErr','Ap','Frametime','Exp','Telapse']
    peak_info = peaklist[peaklist.sn_name == name]
    w2mag_o = np.asarray(sn_data[sn_data.Filter == 'UVW2']['Mag'])
    w2mjd_o = np.asarray(sn_data[sn_data.Filter == 'UVW2']['MJD'])
    w2mage_o = np.asarray(sn_data[sn_data.Filter == 'UVW2']['Magerr'])
    #print(w2mag,len(w2mag),len(w2mjd),w2mjd[6])
    w2mag = []
    w2mjd = []
    w2mage = []
    for i in range(len(w2mag_o)):
        #print(w2mag[i],w2mjd[i])
        if np.isnan(w2mag_o[i]) == False and  np.isnan(w2mjd_o[i]) == False:
            w2mag.append(w2mag_o[i])
            w2mjd.append(w2mjd_o[i])
            w2mage.append(w2mage_o[i])

    if len(w2mag) >= 4 and np.isnan(peak_info.iloc[0]['w2t0']) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        w2phase = (w2mjd-peak_info.iloc[0]['w2t0'])/(1+peak_info.iloc[0]['z'])
        w2check = True
        for k in range(len(w2phase)):
            if w2phase[k] <= 40.0:
                w2stop = k
            else:
                w2stop = k-1
                break 
        for k in range(len(w2phase)):
            if w2phase[k] >= -10.0:
                w2start = k
                break   
    else:
        w2check = False
  

        
    m2mag_o = np.asarray(sn_data[sn_data.Filter == 'UVM2']['Mag'])
    m2mjd_o = np.asarray(sn_data[sn_data.Filter == 'UVM2']['MJD'])
    m2mage_o = np.asarray(sn_data[sn_data.Filter == 'UVM2']['Magerr'])
    m2mag = []
    m2mjd = []
    m2mage = []
    for i in range(len(m2mag_o)):
       
        if np.isnan(m2mag_o[i]) == False and  np.isnan(m2mjd_o[i]) == False:
            m2mag.append(m2mag_o[i])
            m2mjd.append(m2mjd_o[i])
            m2mage.append(m2mage_o[i])
    if len(m2mag) >= 4 and np.isnan(peak_info.iloc[0]['m2t0']) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        m2phase = (m2mjd-peak_info.iloc[0]['m2t0'])/(1+peak_info.iloc[0]['z'])
        m2check = True
        for k in range(len(m2phase)):
            if m2phase[k] <= 40.0:
                m2stop = k
            else:
                m2stop = k-1
                break 
        for k in range(len(m2phase)):
            if m2phase[k] >= -10.0:
                m2start = k
                break 
    else:
        m2check = False
       
    w1mag_o = np.asarray(sn_data[sn_data.Filter == 'UVW1']['Mag'])
    w1mjd_o = np.asarray(sn_data[sn_data.Filter == 'UVW1']['MJD'])
    w1mage_o = np.asarray(sn_data[sn_data.Filter == 'UVW1']['Magerr'])
    w1mag = []
    w1mjd = []
    w1mage = []
    for i in range(len(w1mag_o)):
        if np.isnan(w1mag_o[i]) == False and  np.isnan(w1mjd_o[i]) == False:
            w1mag.append(w1mag_o[i])
            w1mjd.append(w1mjd_o[i])
            w1mage.append(w1mage_o[i])
    if len(w1mag) >= 4 and np.isnan(np.float(peak_info.iloc[0]['w1t0'])) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        w1phase = (np.array(w1mjd)-np.float(peak_info.iloc[0]['w1t0']))/(1+peak_info.iloc[0]['z'])
        w1check = True    
        for k in range(len(w1phase)):
            if w1phase[k] <= 40.0:
                w1stop = k
            else:
                w1stop = k-1
                break 
        for k in range(len(w1phase)):
            if w1phase[k] >= -10.0:
                w1start = k
                break
    else:
        w1check = False  

    umag_o = np.asarray(sn_data[sn_data.Filter == 'U']['Mag'])
    umjd_o = np.asarray(sn_data[sn_data.Filter == 'U']['MJD'])
    umage_o = np.asarray(sn_data[sn_data.Filter == 'U']['Magerr'])
    umag = []
    umjd = []
    umage = []
    for i in range(len(umag_o)):
        if np.isnan(umag_o[i]) == False and  np.isnan(umjd_o[i]) == False:
            umag.append(umag_o[i])
            umjd.append(umjd_o[i])
            umage.append(umage_o[i])
    if len(umag) >= 4 and np.isnan(peak_info.iloc[0]['ut0']) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        uphase = (umjd-peak_info.iloc[0]['ut0'])/(1+peak_info.iloc[0]['z'])
        ucheck = True
        for k in range(len(uphase)):
            if uphase[k] <= 40.0:
                ustop = k
            else:
                ustop = k-1
                break 
        for k in range(len(uphase)):
            if uphase[k] >= -10.0:
                ustart = k
                break
    else:
        ucheck = False
    if name == 'SN2011fe':
        ucheck = False
    elif name == 'SN2005cf':
        ucheck = False
    bmag_o = np.asarray(sn_data[sn_data.Filter == 'B']['Mag'])
    bmjd_o = np.asarray(sn_data[sn_data.Filter == 'B']['MJD'])    
    bmage_o = np.asarray(sn_data[sn_data.Filter == 'B']['Magerr'])
    bmag = []
    bmjd = []
    bmage = []
    for i in range(len(bmag_o)):
        if np.isnan(bmag_o[i]) == False and  np.isnan(bmjd_o[i]) == False:
            bmag.append(bmag_o[i])
            bmjd.append(bmjd_o[i])
            bmage.append(bmage_o[i])
    if len(bmag) >= 4 and np.isnan(peak_info.iloc[0]['bt0']) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        bphase = (bmjd-peak_info.iloc[0]['bt0'])/(1+peak_info.iloc[0]['z'])
        bcheck = True
        for k in range(len(bphase)):
            if bphase[k] <= 40.0:
                bstop = k
            else:
                bstop = k-1
                break 
        for k in range(len(bphase)):
            if bphase[k] >= -11.0: #Normally 10, but SN2012HR has a data point just barely above -10.0 so I raised it
                bstart = k
                break
    else:
        bcheck = False

    vmag_o = np.asarray(sn_data[sn_data.Filter == 'V']['Mag'])
    vmjd_o = np.asarray(sn_data[sn_data.Filter == 'V']['MJD'])
    vmage_o = np.asarray(sn_data[sn_data.Filter == 'V']['Magerr'])
    vmag = []
    vmjd = []
    vmage = []
    for i in range(len(vmag_o)):
        if np.isnan(vmag_o[i]) == False and  np.isnan(vmjd_o[i]) == False:
            vmag.append(vmag_o[i])
            vmjd.append(vmjd_o[i])
            vmage.append(vmage_o[i])
    if len(vmag) >= 4 and np.isnan(peak_info.iloc[0]['vt0']) == False and np.isnan(peak_info.iloc[0]['z']) == False:
        vphase = (vmjd-peak_info.iloc[0]['vt0'])/(1+peak_info.iloc[0]['z'])
        vcheck = True
        for k in range(len(vphase)):
            if vphase[k] <= 40.0:
                vstop = k
            else:
                vstop = k-1
                break 
        for k in range(len(vphase)):
            if vphase[k] >= -10.0:
                vstart = k
                break  
    else:
        vcheck = False
    
            
    plt.figure(fignum,figsize=(9,7),dpi=300)
    plt.subplot(231)
    if w2check == True:
        xnew = w2phase[w2start:w2stop+1]#np.linspace(min(w2phase), max(w2phase), num=len(w2phase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        
        def band(x,a,b,c,d):
            return a*t1 +b*t2 +c*t3 +d+mn
        try:
            poptw2, pcovw2 = curve_fit(band,w2phase[w2start:w2stop+1],w2mag[w2start:w2stop+1],sigma=w2mage[w2start:w2stop+1],
                                      p0=(1,1,1,peak_info.iloc[0]['w2peak']))
        except RuntimeError:
            plt.scatter(0,0)
            finw2.append([name,nana,nana])
            plt.text(-0.02,0.02,'runtime error')
        else:
            xnew = np.linspace(w2phase[w2start], w2phase[w2stop],num=1000,endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            ynew = band(xnew,poptw2[0],poptw2[1],poptw2[2],poptw2[3])
            finw2.append([name,poptw2,np.diag(pcovw2)])
            newmin = np.where(ynew==min(ynew))
            plt.errorbar(w2phase[w2start:w2stop+1]-xnew[newmin],w2mag[w2start:w2stop+1],
                         yerr=w2mage[w2start:w2stop+1],linestyle='none',marker='.')
            
            if abs(float(xnew[newmin])) < 1.2 and  w2phase[w2start] < 0.0 and w2phase[w2stop] > 15.0:
                resi = 1
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + peak_info.iloc[0]['w2t0']
                
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovw2[0][0]) + (t2**2 * pcovw2[1][1]) + (t3**2 * pcovw2[2][2]) 
                + 2*t1*t2*pcovw2[0][1]+ 2*t1*t3*pcovw2[0][2]+ 2*t3*t2*pcovw2[1][2] 
                          + pcovw2[3][3]+ 2*t1*pcovw2[0][3]+ 2*t2*pcovw2[1][3]+ 2*t3*pcovw2[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptw2[0],poptw2[1],poptw2[2],poptw2[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovw2[0][0]) + (t2**2 * pcovw2[1][1]) + (t3**2 * pcovw2[2][2]) 
                    + 2*t1*t2*pcovw2[0][1]+ 2*t1*t3*pcovw2[0][2]+ 2*t3*t2*pcovw2[1][2]
                          + pcovw2[3][3]+ 2*t1*pcovw2[0][3]+ 2*t2*pcovw2[1][3]+ 2*t3*pcovw2[2][3])
                onesig = np.sqrt(onesig)
            
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)
            else:
                resi = 0
                dm = 0
                mjd = 0.0
                edm=0
            dfw2 = dfw2.append({'SNname':name,'weight1':poptw2[0],'weight2':poptw2[1],'weight3':poptw2[2],
                                'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                                'dM15':float(dm),'resid':resi},ignore_index=True)   
            efw2 = efw2.append({'SNname':name,'pcov':np.diag(pcovw2),'dmstd':edm},ignore_index=True)
    else:
        plt.scatter(0,0)
        finw2.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data  ')
    plt.title('W2')
    plt.ylabel('Apparent Magnitude')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')
    
    plt.subplot(232)
    if m2check == True:
        xnew = m2phase[m2start:m2stop+1]#np.linspace(min(m2phase), max(m2phase), num=len(m2phase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        def band(x,a,b,c,d):
            return a*t1 +b*t2 +c*t3  +d+mn
        try:
            poptm2, pcovm2 = curve_fit(band,m2phase[m2start:m2stop+1],m2mag[m2start:m2stop+1],sigma=m2mage[m2start:m2stop+1]
                                      ,p0=(1,1,1,peak_info.iloc[0]['m2peak']))
        except RuntimeError:
            plt.scatter(0,0)
            finm2.append([name,nana,nana])
            plt.text(-0.02,0.02,'runtime error')
        else:
            xnew = np.linspace(m2phase[m2start], m2phase[m2stop],num=1000,endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            ynew = band(xnew,poptm2[0],poptm2[1],poptm2[2],poptm2[3])
            newmin = np.where(ynew==min(ynew))
            plt.errorbar(m2phase[m2start:m2stop+1]-xnew[newmin],m2mag[m2start:m2stop+1],
                         yerr=m2mage[m2start:m2stop+1],linestyle='none',marker='.')
            
            if  abs(float(xnew[newmin])) < 1.2 and m2phase[m2start] < 0.0 and m2phase[m2stop] > 15.0:
                resi = 1
                print(xnew[newmin])
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + peak_info.iloc[0]['m2t0']
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovm2[0][0]) + (t2**2 * pcovm2[1][1]) + (t3**2 * pcovm2[2][2]) 
                + 2*t1*t2*pcovm2[0][1]+ 2*t1*t3*pcovm2[0][2]+ 2*t3*t2*pcovm2[1][2]
                         + pcovm2[3][3]+ 2*t1*pcovm2[0][3]+ 2*t2*pcovm2[1][3]+ 2*t3*pcovm2[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptm2[0],poptm2[1],poptm2[2],poptm2[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovm2[0][0]) + (t2**2 * pcovm2[1][1]) + (t3**2 * pcovm2[2][2]) 
                    + 2*t1*t2*pcovm2[0][1]+ 2*t1*t3*pcovm2[0][2]+ 2*t3*t2*pcovm2[1][2]
                          + pcovm2[3][3]+ 2*t1*pcovm2[0][3]+ 2*t2*pcovm2[1][3]+ 2*t3*pcovm2[2][3])
                onesig = np.sqrt(onesig)
            
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)

            else:
                resi = 0
                dm = 0
                mjd=0.0
                edm = 0
            dfm2 = dfm2.append({'SNname':name,'weight1':poptm2[0],'weight2':poptm2[1],'weight3':poptm2[2],
                                'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                                'dM15':float(dm),'resid':resi},ignore_index=True)   
            efm2 = efm2.append({'SNname':name,'pcov':np.diag(pcovm2),'dmstd':edm},ignore_index=True)
            finm2.append([name,poptm2,np.diag(pcovm2)])
    else:
        plt.scatter(0,0)
        finm2.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data  ')
    plt.title('M2')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')
    
    plt.subplot(233)
    if w1check == True:
        xnew = w1phase[w1start:w1stop+1]#np.linspace(min(w1phase), max(w1phase), num=len(w1phase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        def band(x,a,b,c,d):
            return a*t1 +b*t2 +c*t3 + d+mn
        try:
            poptw1, pcovw1 = curve_fit(band,w1phase[w1start:w1stop+1],w1mag[w1start:w1stop+1],sigma=w1mage[w1start:w1stop+1],
                                       p0=(1,1,1,peak_info.iloc[0]['w1peak']))
        except RuntimeError:
            plt.scatter(0,0)
            finw1.append([name,nana,nana])
            plt.text(-0.02,0.02,'runtime error')
        else:
 
            xnew = np.linspace(w1phase[w1start], w1phase[w1stop],num=1000,endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            #plt.plot(xnew,band(xnew,poptw1[0],poptw1[1],poptw1[2],poptw1[3]))
            ynew = band(xnew,poptw1[0],poptw1[1],poptw1[2],poptw1[3])
            newmin = np.where(ynew==min(ynew))
            plt.errorbar(w1phase[w1start:w1stop+1]-xnew[newmin],w1mag[w1start:w1stop+1],
                         yerr=w1mage[w1start:w1stop+1],linestyle='none',marker='.')
            
            if abs(float(xnew[newmin])) < 1.2 and  w1phase[w1start] < 0.0 and w1phase[w1stop] > 15.0:
                resi = 1
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + np.float(peak_info.iloc[0]['w1t0'])
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovw1[0][0]) + (t2**2 * pcovw1[1][1]) + (t3**2 * pcovw1[2][2]) 
                + 2*t1*t2*pcovw1[0][1]+ 2*t1*t3*pcovw1[0][2]+ 2*t3*t2*pcovw1[1][2]
                         + pcovw1[3][3]+ 2*t1*pcovw1[0][3]+ 2*t2*pcovw1[1][3]+ 2*t3*pcovw1[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptw1[0],poptw1[1],poptw1[2],poptw1[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovw1[0][0]) + (t2**2 * pcovw1[1][1]) + (t3**2 * pcovw1[2][2]) 
                    + 2*t1*t2*pcovw1[0][1]+ 2*t1*t3*pcovw1[0][2]+ 2*t3*t2*pcovw1[1][2]
                          + pcovw1[3][3]+ 2*t1*pcovw1[0][3]+ 2*t2*pcovw1[1][3]+ 2*t3*pcovw1[2][3])
                onesig = np.sqrt(onesig)
            
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)
            else:
                resi = 0
                dm = 0
                mjd=0.0
                edm = 0
            dfw1 = dfw1.append({'SNname':name,'weight1':poptw1[0],'weight2':poptw1[1],'weight3':poptw1[2],
                                'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                                'dM15':float(dm),'resid':resi},ignore_index=True)   
            efw1 = efw1.append({'SNname':name,'pcov':np.diag(pcovw1),'dmstd':edm},ignore_index=True)
            finw1.append([name,poptw1,np.diag(pcovw1)])
    else:
        plt.scatter(0,0)
        finw1.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data  ')
    plt.title('W1')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')

    plt.subplot(234)
    if ucheck == True:
        xnew = uphase[ustart:ustop+1]#np.linspace(min(uphase), max(uphase), num=len(uphase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        def band(x,a,b,c,d):
            return a*t1  +b*t2 +c*t3  +d+mn
        try:
            poptu, pcovu = curve_fit(band,uphase[ustart:ustop+1],umag[ustart:ustop+1],sigma=umage[ustart:ustop+1],
                                    p0=(1,1,1,peak_info.iloc[0]['upeak']))
        except RuntimeError:
            plt.scatter(0,0)
            finu.append([name,nana,nana])
            plt.text(-0.02,0.02,'runtime error')
        else:
  
            xnew = np.linspace(uphase[ustart], uphase[ustop], num=1000, endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            #plt.plot(xnew,band(xnew,poptu[0],poptu[1],poptu[2],poptu[3]))
            ynew = band(xnew,poptu[0],poptu[1],poptu[2],poptu[3])
            newmin = np.where(ynew==min(ynew))
            plt.errorbar(uphase[ustart:ustop+1]-xnew[newmin],umag[ustart:ustop+1],
                         yerr=umage[ustart:ustop+1],linestyle='none',marker='.')
            
            if  abs(float(xnew[newmin])) < 1.2 and uphase[ustart] < 0.0 and uphase[ustop] > 15.0:
                resi = 1
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + peak_info.iloc[0]['ut0']
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovu[0][0]) + (t2**2 * pcovu[1][1]) + (t3**2 * pcovu[2][2]) 
                + 2*t1*t2*pcovu[0][1]+ 2*t1*t3*pcovu[0][2]+ 2*t3*t2*pcovu[1][2]
                         + pcovu[3][3]+ 2*t1*pcovu[0][3]+ 2*t2*pcovu[1][3]+ 2*t3*pcovu[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptu[0],poptu[1],poptu[2],poptu[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovu[0][0]) + (t2**2 * pcovu[1][1]) + (t3**2 * pcovu[2][2]) 
                    + 2*t1*t2*pcovu[0][1]+ 2*t1*t3*pcovu[0][2]+ 2*t3*t2*pcovu[1][2]
                          + pcovu[3][3]+ 2*t1*pcovu[0][3]+ 2*t2*pcovu[1][3]+ 2*t3*pcovu[2][3])
                onesig = np.sqrt(onesig)
            
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)
            else:
                resi = 0
                dm = 0
                mjd=0.0
                edm = 0
            dfu = dfu.append({'SNname':name,'weight1':poptu[0],'weight2':poptu[1],'weight3':poptu[2],
                              'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                              'dM15':float(dm),'resid':resi},ignore_index=True)   
            efu = efu.append({'SNname':name,'pcov':np.diag(pcovu),'dmstd':edm},ignore_index=True)
            finu.append([name,poptu,np.diag(pcovu)])
    else:
        plt.scatter(0,0)
        finu.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data  ')
    plt.title('U')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')
    
    plt.subplot(235)
    if bcheck == True:
        xnew = bphase[bstart:bstop+1]#np.linspace(min(bphase), max(bphase), num=len(bphase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        def band(x,a,b,c,d):
            return a*t1 +b*t2 +c*t3  +d+mn
        try:
            poptb, pcovb = curve_fit(band,bphase[bstart:bstop+1],bmag[bstart:bstop+1],sigma=bmage[bstart:bstop+1],
                                    p0=(1,1,1,peak_info.iloc[0]['bpeak']))
        except RuntimeError:
            plt.scatter(0,0)
            finb.append([name,nana,nana])
            plt.text(-0.02,0.02,'runtime error')
        else:
 
            xnew = np.linspace(bphase[bstart],bphase[bstop], num=1000, endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            #plt.plot(xnew,band(xnew,poptb[0],poptb[1],poptb[2],poptb[3]))
            ynew = band(xnew,poptb[0],poptb[1],poptb[2],poptb[3])
            newmin = np.where(ynew==min(ynew)) 
            plt.errorbar(bphase[bstart:bstop+1]-xnew[newmin],bmag[bstart:bstop+1],
                         yerr=bmage[bstart:bstop+1],linestyle='none',marker='.')
            
            if  abs(float(xnew[newmin])) < 1.2 and bphase[bstart] < 0.0 and bphase[bstop] > 15.0:
                resi = 1
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + peak_info.iloc[0]['bt0']
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovb[0][0]) + (t2**2 * pcovb[1][1]) + (t3**2 * pcovb[2][2]) 
                + 2*t1*t2*pcovb[0][1]+ 2*t1*t3*pcovb[0][2]+ 2*t3*t2*pcovb[1][2]
                         + pcovb[3][3]+ 2*t1*pcovb[0][3]+ 2*t2*pcovb[1][3]+ 2*t3*pcovb[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptb[0],poptb[1],poptb[2],poptb[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovb[0][0]) + (t2**2 * pcovb[1][1]) + (t3**2 * pcovb[2][2]) 
                    + 2*t1*t2*pcovb[0][1]+ 2*t1*t3*pcovb[0][2]+ 2*t3*t2*pcovb[1][2]
                          + pcovb[3][3]+ 2*t1*pcovb[0][3]+ 2*t2*pcovb[1][3]+ 2*t3*pcovb[2][3])
                onesig = np.sqrt(onesig)
                
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)
            else:
                resi = 0
                dm = 0
                mjd=0.0
                edm = 0
            dfb = dfb.append({'SNname':name,'weight1':poptb[0],'weight2':poptb[1],'weight3':poptb[2],
                              'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                              'dM15':float(dm),'resid':resi},ignore_index=True)   
            efb = efb.append({'SNname':name,'pcov':np.diag(pcovb),'dmstd':edm},ignore_index=True)
            
            finb.append([name,poptb,np.diag(pcovb)])
    else:
        plt.scatter(0,0)
        finb.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data  ')
    plt.title('B')
    plt.xlabel('Phase Since Band Peak')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')
    
    plt.subplot(236)
    if vcheck == True:
        xnew =vphase[vstart:vstop+1]# np.linspace(min(vphase), max(vphase), num=len(vphase), endpoint=True)
        mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
        def band(x,a,b,c,d):
            return a*t1  +b*t2 +c*t3  + d+mn
        try:
            poptv, pcovv = curve_fit(band,vphase[vstart:vstop+1],vmag[vstart:vstop+1],sigma=vmage[vstart:vstop+1],
                                    p0=(1,1,1,peak_info.iloc[0]['vpeak']))
        except RuntimeError:
            plt.scatter(0,0)
            finv.append([name,nana,nana])  
            plt.text(-0.02,0.02,'runtime error')
        else:
        
            xnew = np.linspace(vphase[vstart], vphase[vstop], num=1000, endpoint=True)
            mn,t1,t2,t3,t4 = means(xnew),temp1(xnew),temp2(xnew),temp3(xnew),temp4(xnew)
            #plt.plot(xnew,band(xnew,poptv[0],poptv[1],poptv[2],poptv[3]))
            ynew = band(xnew,poptv[0],poptv[1],poptv[2],poptv[3])
            newmin = np.where(ynew==min(ynew))
            plt.errorbar(vphase[vstart:vstop+1]-xnew[newmin],vmag[vstart:vstop+1],
                         yerr=vmage[vstart:vstop+1],linestyle='none',marker='.')
            
            if abs(float(xnew[newmin])) < 1.2 and vphase[vstart] < 0.0 and vphase[vstop] > 15.0:
                resi = 1
                mjd = (1+peak_info.iloc[0]['z']) * float(xnew[newmin]) + peak_info.iloc[0]['vt0']
                plt.plot(xnew-xnew[newmin],ynew  ,c='orange',linestyle='--')
                onesig = ((t1**2 * pcovv[0][0]) + (t2**2 * pcovv[1][1]) + (t3**2 * pcovv[2][2]) 
                + 2*t1*t2*pcovv[0][1]+ 2*t1*t3*pcovv[0][2]+ 2*t3*t2*pcovv[1][2]
                         + pcovv[3][3]+ 2*t1*pcovv[0][3]+ 2*t2*pcovv[1][3]+ 2*t3*pcovv[2][3])
                onesig = np.sqrt(onesig)
                plt.plot(xnew-xnew[newmin],ynew-onesig ,c='red',linestyle='--')
                plt.plot(xnew-xnew[newmin],ynew+onesig  ,c='red',linestyle='--')
                plt.fill_between(xnew-xnew[newmin], ynew-onesig, ynew+onesig,facecolor='gray',alpha=0.5)
                
                xnew2 = np.linspace(0,15,num=1000,endpoint=True)
                mn,t1,t2,t3,t4 = means(xnew2),temp1(xnew2),temp2(xnew2),temp3(xnew2),temp4(xnew2)
                ynew2 = band(xnew2,poptv[0],poptv[1],poptv[2],poptv[3])
                newmin2 = np.where(ynew==min(ynew))
                onesig = ((t1**2 * pcovv[0][0]) + (t2**2 * pcovv[1][1]) + (t3**2 * pcovv[2][2]) 
                    + 2*t1*t2*pcovv[0][1]+ 2*t1*t3*pcovv[0][2]+ 2*t3*t2*pcovv[1][2]
                          + pcovv[3][3]+ 2*t1*pcovv[0][3]+ 2*t2*pcovv[1][3]+ 2*t3*pcovv[2][3])
                onesig = np.sqrt(onesig)
            
                dm = round(ynew2[-1]-ynew2[0],3)
                edm = round((onesig[0]**2 + onesig[-1]**2)**0.5,3)
            else:
                resi = 0
                dm = 0
                mjd=0.0
                edm = 0
            dfv = dfv.append({'SNname':name,'weight1':poptv[0],'weight2':poptv[1],'weight3':poptv[2],
                              'peak_mjd':mjd,'peak_mag':min(ynew),'peak_phase':float(xnew[newmin]),
                              'dM15':float(dm),'resid':resi},ignore_index=True)   
            efv = efv.append({'SNname':name,'pcov':np.diag(pcovv),'dmstd':edm},ignore_index=True)
            
            finv.append([name,poptv,np.diag(pcovv)])
    else:
        plt.scatter(0,0)
        finv.append([name,nana,nana])
        plt.text(-0.02,0.02,'not enough data')
    plt.title('V')
    plt.gca().invert_yaxis()
    plt.minorticks_on()
    plt.tick_params(which='both',direction='in')
    
    plt.tight_layout()
    plt.savefig('snfits_p2/'+name+'_bandvague.png',facecolor='white')
    #plt.show()
    plt.close()

In [4]:
bv('SN2022hrs',1)
print('UVW2')
print(dfw2)
print(' ')
print('UVM2')
print(dfm2)
print(' ')
print('UVW1')
print(dfw1)
print(' ')
print('U')
print(dfu)
print(' ')
print('B')
print(dfb)
print(' ')
print('V')
print(dfv)
print(' ')

SN2022hrs
UVW2
      SNname   weight1   weight2   weight3      peak_mjd   peak_mag  \
0  SN2022hrs -6.509955  0.848757 -2.415958  59697.442004  15.886833   

   peak_phase   dM15 resid  
0    0.000501  0.991     1  
 
UVM2
      SNname    weight1   weight2    weight3  peak_mjd   peak_mag  peak_phase  \
0  SN2022hrs -34.691312  8.592401 -12.895036       0.0  17.678095    5.401534   

   dM15 resid  
0   0.0     0  
 
UVW1
      SNname   weight1   weight2   weight3      peak_mjd   peak_mag  \
0  SN2022hrs -7.575359 -0.335464 -2.199505  59697.437821  14.353017   

   peak_phase   dM15 resid  
0    0.000718  1.402     1  
 
U
      SNname   weight1  weight2   weight3      peak_mjd   peak_mag  \
0  SN2022hrs -8.078705 -0.01971 -1.998602  59697.439141  12.852357   

   peak_phase   dM15 resid  
0    0.001235  1.381     1  
 
B
      SNname   weight1   weight2   weight3      peak_mjd   peak_mag  \
0  SN2022hrs -4.326023 -0.584509  1.467005  59703.973733  12.991306   

   peak_phase  dM15 resi

In [ ]:
for i in range(len(peaklist)):
    mane = peaklist.iloc[i]['sn_name']
    if path.exists('sndata_files/'+mane+'_uvotB15.1.dat') == True:
        bv(mane,i)
    elif path.exists('sndata_files/'+mane+'_uvotB22.1.dat') == True:
        bv(mane,i)
    elif path.exists('sndata_files/'+mane+'_uvotB22.18.dat') == True:
        bv(mane,i)
    else:
        print(mane+' no such file')

ASASSN-15ti
ASASSN-15so
[0.09261834]
ASASSN-15sf


/Users/yaswantd/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


SN2015ar
[0.09811355]
ASASSN-15rw
ASASSN-15rq
PSNJ09100885+5003396 no such file
SN2015ak
ASASSN-15od
SN2015M
ASASSN-15kg
ASASSN-15hy no such file
SN2015F
[0.06970732]
SN2015bp
SN2015bo
ASASSN-15ar
iPTF14atg
ASASSN-14lw
[0.09804007]
ASASSN-14lp
[0.097498]
SN2014eg
CSS140914
iPTF14bdn
[0.1145158]
PS1-14xw
SN2014ap
SN2014ai
SN2014J
[0.0972127]
iPTF13ebh
iPTF13dge
LSQ13ry


/Users/yaswantd/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


SN2013gy
SN2013gs
SN2013gh
SN2013fw
SN2013ex
[-0.0149609]
SN2013dy
SN2013cs
[0.11071438]
SN2013cg
SN2013bs


...

SN2013ag
SN2013aa
[0.08775219]
SN2013Q
SN2013E
LSQ12gdj
[0.09395405]
SN2012ij
SN2012ht
[0.07343928]
SN2012hr
[0.00918389]
SN2012fr
[0.08183497]
SN2012dn
[-0.00385871]
SN2012cg
[0.07527928]
SN2012Z
SN2012V
[0.08555885]
SN2011iv
[0.00811915]
SN2011im
SN2011ia
[0.09953496]
SN2011hb
SN2011fe
[0.08692329]
SN2011ek
SN2011eh
SN2011dn
SN2011dm
SN2011de
SN2011by
[0.05904728]
SN2011ao
[0.10492382]
SN2011aa
[0.63187614]
SN2011M
SN2011B
[0.01764571]
SN2010ko
SN2010kg
SN2010gp
SN2010gn
SN2010ev


/Users/yaswantd/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


SN2010cr
SN2010Y
[0.00674215]
PTF10icb
SN2009jb
SN2009ig
[0.09115488]
SN2009cz
SN2009an
SN2009Y
SNF20080514002
[0.2905947]
SN2008hv
[0.02688325]
SN2008hs
SN2008ec
SN2008Q
[-0.01167954]
SN2007on
[0.00772691]
SN2007cv
SN2007cq
SN2007co


/Users/yaswantd/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


SN2007af
[0.02677334]
SN2007S
SN2006mr


In [ ]:
dfw2.to_csv('snfits_p2/w2data.csv',index=False)
dfm2.to_csv('snfits_p2/m2data.csv',index=False)
dfw1.to_csv('snfits_p2/w1data.csv',index=False)
dfu.to_csv('snfits_p2/udata.csv',index=False)
dfb.to_csv('snfits_p2/bdata.csv',index=False)
dfv.to_csv('snfits_p2/vdata.csv',index=False)

efw2.to_csv('snfits_p2/w2err.csv',index=False)
efm2.to_csv('snfits_p2/m2err.csv',index=False)
efw1.to_csv('snfits_p2/w1err.csv',index=False)
efu.to_csv('snfits_p2/uerr.csv',index=False)
efb.to_csv('snfits_p2/berr.csv',index=False)
efv.to_csv('snfits_p2/verr.csv',index=False)


In [ ]:
dfw2